## 3. 한글 문서의 분류
다음무비(http://movie.daum.net)로부터 crawl한 영화리뷰를 이용하여 분류 연습<br>
영화리뷰와 영화의 제목을 학습해서 주어진 리뷰내용으로 어떤 영화에 대한 리뷰인지를 예측하고자 함
### data file 내용
'신과함께', '코코', '라라랜드', '인피니티 워', '곤지암' 다섯개의 영화에 대해 총 1827개의 리뷰를 수집
csv 파일 안에 리뷰내용, 평점, 영화이름 의 순으로 저장되어 있음

In [1]:
import csv

text = []
y = []
with open('movie_data.csv', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        #print(row)
        if row: #그 줄에 내용이 있는 경우에만
            text.append(row[0]) #영화 리뷰를 text 리스트에 추가
            y.append(row[2]) #영화이름을 text 리스트에 추가

In [2]:
print('Num of samples: {}'.format(len(text)))
print('Movie titles of reivews: {}'.format(set(y)))

Num of samples: 3653
Movie titles of reivews: {'', '신과함께', '인피니티 워', '라라랜드', '코코', '곤지암'}


In [3]:
from sklearn.model_selection import train_test_split

# split data and labels into a training and a test set
X_train, X_test, y_train, y_test = train_test_split(text, y, random_state=0)
# 비율을 지정하지 않으면 75:25로 분할됨

In [4]:
len(X_train) #1827의 0.75

2739

In [5]:
from konlpy.tag import Okt #konlpy에서 Twitter 형태소 분석기를 import
#from konlpy.tag import Twitter #konlpy에서 Twitter 형태소 분석기를 import
twitter_tag = Okt()
#twitter_tag = Twitter()

In [6]:
print(twitter_tag.morphs(X_train[1])) #둘째 리뷰에 대해 형태소 단위로 tokenize

['ㅜ', 'ㅜ']


In [7]:
twitter_tag.nouns(X_train[1]) #둘째 리뷰에서 명사만 추출

[]

In [8]:
def twit_tokenizer(text): # Twitter 형태소 분석기의 명사추출함수를 tokenizer 함수로 사용
    return twitter_tag.nouns(text)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=2) #Twitter 형태소분석기에서 명사만 추출하는 함수를 tokenizer로 이용
# twit_tokenizer 대신 twitter_tag.nouns를 직접 써도 됨
# 하나의 문서에서만 출현한 단어는 쓸모가 없으므로 제외, 즉 최소 document frequency를 2로 설정

X_train_tfidf = tfidf.fit_transform(X_train) # train data 변환 -> tfidf vector
X_test_tfidf = tfidf.transform(X_test) # test data 변환 -> tfidf vector

clf = LogisticRegression() # logistic regression 분류기 선언
clf.fit(X_train_tfidf, y_train) # 분류기 학습
print('Train score', clf.score(X_train_tfidf, y_train)) # train data 예측정확도
print('Test score', clf.score(X_test_tfidf, y_test)) # test data 예측정확도
print(X_train_tfidf.shape) # 총 1156개의 명사로 이루어짐

Train score 0.8324205914567361
Test score 0.7286652078774617
(2739, 1180)


In [10]:
X_test[:10] #test data에서 앞 10개를 출력

['',
 '꼭봐야함 진심',
 '서릿발 같은 원한과 복수가 없으니 더 서늘하다.',
 '',
 '이 정도면,,,울 나라 영화 치고  상당히 잘 만들었다 생각한다.  몽조리,,허리웃에,,맛들어가지고,,참,, 평점이 너무 박하다.  나름,,공포지수는 상당했다.  빼어난 수작이다.   평점은,,,7,8점대나,,너무 박해서,,,10점 준다.  이상.',
 '예고편인줄 알면 안보러 갔을것',
 '',
 '역대급이었습니다 다음 2편 후속작도 기대하겠습니다',
 '',
 '아름답고, 따듯한 사후세계. 신과함께 보고 걸린 암이 나았습니다.']

In [11]:
clf.predict(X_test_tfidf[:10]) # test data의 앞 10개에 대한 예측내용

array(['', '', '인피니티 워', '', '곤지암', '', '', '인피니티 워', '', ''], dtype='<U6')

In [13]:
print(y_test[:10]) # test data 앞 10개의 실제 영화제목

['', '인피니티 워', '곤지암', '', '곤지암', '인피니티 워', '', '인피니티 워', '', '코코']


### 성능을 개선하기 위한 노력

In [14]:
# morphs()는 명사 외에도 모든 형태소를 포함
print(twitter_tag.morphs(X_train[1]))

['ㅜ', 'ㅜ']


In [15]:
tfidf = TfidfVectorizer(tokenizer=twitter_tag.morphs, min_df=2) # 명사 대신 모든 형태소를 사용
#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
print('Train score', clf.score(X_train_tfidf, y_train))
print('Test score', clf.score(X_test_tfidf, y_test))
print(X_train_tfidf.shape)
#명사만 사용한 것에 비해 train score는 상승, test score는 하락

Train score 0.8711208470244615
Test score 0.7286652078774617
(2739, 2302)


In [16]:
print(twitter_tag.pos(X_train[1], norm=True, stem=True)) #pos()는 형태소와 품사를 함께 제공

[('ㅜ', 'KoreanParticle'), ('ㅜ', 'KoreanParticle')]


In [17]:
def twit_tokenizer2(text): #전체를 다 사용하는 대신, 명사, 동사, 형용사를 사용
    target_tags = ['Noun', 'Verb', 'Adjective']
    result = []
    for word, tag in twitter_tag.pos(text, norm=True, stem=True):
        if tag in target_tags:
            result.append(word)
#            result.append('/'.join([word, tag]))
    return result

In [18]:
print(twit_tokenizer2(X_train[1])) # 사용 예

[]


In [19]:
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer2, min_df=2) #명사, 동사, 형용사를 이용하여 tfidf 생성
#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
print('Train score', clf.score(X_train_tfidf, y_train))
print('Test score', clf.score(X_test_tfidf, y_test))
print(X_train_tfidf.shape)
# 현재까지 중에서 test score가 가장 뛰어남

Train score 0.8652792990142387
Test score 0.7450765864332604
(2739, 1598)


In [20]:
# 모든 형태소를 다 사용하고 품사를 알 수 있도록 하면?
def twit_tokenizer3(text):
    #target_tags = ['Noun', 'Verb', 'Adjective']
    result = []
    for word, tag in twitter_tag.pos(text, norm=True, stem=True):
        result.append('/'.join([word, tag])) #단어의 품사를 구분할 수 있도록 함
    return result

In [21]:
tfidf = TfidfVectorizer(tokenizer=twit_tokenizer3, min_df=2)
#tfidf = TfidfVectorizer(tokenizer=twit_tokenizer, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression()
clf.fit(X_train_tfidf, y_train)
print('Train score', clf.score(X_train_tfidf, y_train))
print('Test score', clf.score(X_test_tfidf, y_test))
print(X_train_tfidf.shape)
#성능이 오히려 떨어지고 품사 표시 없이 전체를 다 사용한 경우에 비해 train은 떨어지고, test는 올라감

Train score 0.8769623950346842
Test score 0.7461706783369803
(2739, 2034)


In [22]:
# train score가 높으므로 ridge를 쓰면 어떨까?
from sklearn.linear_model import RidgeClassifier
ridge_clf = RidgeClassifier(alpha = 1)
ridge_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))
# train score가 올라가는 현상이 벌어짐
# test score가 올라갔으나 명사, 형용사, 동사를 쓴 것보다 떨어짐

Train set score: 0.959
Test set score: 0.767


In [23]:
#lasso를 쓰면?
from sklearn.linear_model import LogisticRegression
import numpy as np
lasso_clf = LogisticRegression(penalty='l1', solver='liblinear')
lasso_clf.fit(X_train_tfidf, y_train)
print('Train set score: {:.3f}'.format(lasso_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(lasso_clf.score(X_test_tfidf, y_test)))
print('Used features count: {}'.format(np.sum(lasso_clf.coef_ != 0)), 'out of', X_train_tfidf.shape[1])

Train set score: 0.834
Test set score: 0.751
Used features count: 375 out of 2034


In [24]:
#lsa를 쓰면?
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=239, n_iter=7, random_state=42) #압축할 component의 수 지정
svd.fit(X_train_tfidf)  
print(svd.explained_variance_ratio_)  #계산된 각 component가 설명하는 분산의 비율
print(svd.explained_variance_ratio_.sum())  #선택된 component들이 설명하는 분산의 합 -> 선택한 component의 수에 따라 달라짐
print(svd.singular_values_) 
print(svd.components_.shape)

[0.0288389  0.01328703 0.01243155 0.01200763 0.01084746 0.00947385
 0.00902128 0.00844526 0.00826019 0.0076784  0.00728165 0.00679249
 0.00664239 0.00634816 0.00602749 0.00584802 0.00581584 0.00560971
 0.00549236 0.00527034 0.00515977 0.00505914 0.00499327 0.00480125
 0.00478183 0.00454265 0.00450766 0.00439897 0.00431664 0.00425505
 0.00421284 0.00417336 0.00411537 0.00405579 0.00397664 0.00392832
 0.0039006  0.00380968 0.00379675 0.00377991 0.0037198  0.00365839
 0.00358171 0.00357179 0.00354034 0.00347207 0.00343077 0.00337938
 0.00333398 0.00329826 0.00327249 0.00321609 0.00321032 0.00315627
 0.00310886 0.00308762 0.00307204 0.0030458  0.00301332 0.0029981
 0.00296195 0.00293069 0.0028688  0.00285055 0.0028344  0.00281693
 0.00278754 0.00275978 0.00274434 0.00274162 0.00270626 0.00267445
 0.00266058 0.00261217 0.00260722 0.00259654 0.00257639 0.00253159
 0.0025145  0.00250239 0.00249813 0.00245706 0.00244653 0.00243859
 0.00243555 0.00242538 0.00237339 0.00236423 0.00235229 0.00233

In [25]:
X_train_svd = svd.transform(X_train_tfidf) #선택된 component를 이용하여 2,000개의 feature로부터 feature extract (dimension reduce)
X_test_svd = svd.transform(X_test_tfidf)

from sklearn.linear_model import LogisticRegression
SVD_clf = LogisticRegression()
SVD_clf.fit(X_train_svd, y_train)
print('Train set score: {:.3f}'.format(SVD_clf.score(X_train_svd, y_train)))
print('Test set score: {:.3f}'.format(SVD_clf.score(X_test_svd, y_test)))

Train set score: 0.834
Test set score: 0.745


In [26]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(tokenizer=twit_tokenizer2, min_df=2).fit(X_train) #tfidf와 동일하게 max_feature를 제한하여 학습
X_train_cv = cv.transform(X_train) # train set을 변환
print('Train set dimension:', X_train_cv.shape) # 36310 대신 2000이 된 것을 확인
X_test_cv = cv.transform(X_test) # test set을 변환
print('Test set dimension:', X_test_cv.shape)

from sklearn.naive_bayes import MultinomialNB
NB_clf = MultinomialNB()
NB_clf.fit(X_train_cv, y_train)
print('Train set score: {:.3f}'.format(NB_clf.score(X_train_cv, y_train)))
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_cv, y_test)))

C:\Users\chaeyeon\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


Train set dimension: (2739, 1598)
Test set dimension: (914, 1598)
Train set score: 0.884
Test set score: 0.790
